# Guard as Actions

This guide will teach you how to use a `Guard` as an action inside a guardrails configuration. 

In [22]:
# Init: remove any existing configuration
!rm -r config
!mkdir config

## Prerequisites

Set up an OpenAI API key, if not already set.

In [23]:
!export OPENAI_API_KEY=$OPENAI_API_KEY    # Replace with your own key

If you're running this inside a notebook, you also need to patch the AsyncIO loop.

In [24]:
import nest_asyncio

nest_asyncio.apply()

## Sample Runnable

Let's create a sample Guard that can detect PII.

In [25]:
# First, install guardrails
!pip install guardrails-ai


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [26]:
from guardrails import Guard, install
try:
    from guardrails.hub import DetectPII
except ImportError:
    install("hub://guardrails/detect_pii")
    from guardrails.hub import DetectPII

g = Guard().use(DetectPII("pii"))

print(g.validate("My name is John Doe"))

/home/zayd/workspace/NeMo-Guardrails/docs/user_guides/guardrails-ai/guard-as-action/.venv/lib/python3.11/site-packages/guardrails/validator_service/__init__.py:85: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(


ValidationOutcome(
    call_id='136926067541648',
    raw_llm_output='My name is John Doe',
    validation_summaries=[
        ValidationSummary(
            validator_name='DetectPII',
            validator_status='fail',
            property_path='$',
            failure_reason='The following text in your response contains PII:\nMy name is John Doe',
            error_spans=[
                ErrorSpan(start=11, end=19, reason='PII detected in John Doe')
            ]
        )
    ],
    validated_output='My name is John Doe',
    reask=None,
    validation_passed=False,
    error=None
)


## Guardrails Configuration 

We'll create a Guard that uses a detect_pii action and flow. Since we're calling our guard "pii_guard", we'll use "pii_guard_validate" in order to see if the LLM output is safe.

In [27]:
%%writefile config/rails.co

define flow detect_pii
  $is_output_safe = execute pii_guard_validate(text=$user_message)

  if not $is_output_safe
    bot refuse to respond
    stop


Writing config/rails.co


In [28]:
%%writefile config/config.yml
models:
 - type: main
   engine: openai
   model: gpt-3.5-turbo-instruct

rails:
  input:
    flows:
      - detect_pii

Writing config/config.yml


To hook the Guardrails AI guard up so that it can be read from Colang, we use the integration's `register_guardrails_guard_actions` function.
This takes a name and registers two actions:

1. [guard_name]_validate: This action is used to detect validation failures in outputs
2. [guard name]_fix: This action is used to automatically fix validation failures in outputs, when possible

In [29]:
from nemoguardrails import RailsConfig, LLMRails
from nemoguardrails.integrations.guardrails_ai.guard_actions import register_guardrails_guard_actions

config = RailsConfig.from_path("./config")
rails = LLMRails(config)

register_guardrails_guard_actions(rails, g, "pii_guard")

## Testing

Let's give this a try. If we invoke the guardrails configuration with a message that contains the "proprietary" keyword, the returned response is "I'm sorry, I can't respond to that".

In [30]:
response = rails.generate("Give me some proprietary information.")
print(response)

/home/zayd/workspace/NeMo-Guardrails/docs/user_guides/guardrails-ai/guard-as-action/.venv/lib/python3.11/site-packages/guardrails/validator_service/__init__.py:85: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(


I'm sorry, but I am not able to provide any proprietary information as it is confidential and cannot be shared with anyone outside of the company. However, I can help you with any other non-proprietary information or tasks you may have. Is there something else I can assist you with?


On the other hand, a message which does not hit the input rail, will proceed as usual.

In [31]:
response = rails.generate("What is the result for 2+2?")
print(response)

/home/zayd/workspace/NeMo-Guardrails/docs/user_guides/guardrails-ai/guard-as-action/.venv/lib/python3.11/site-packages/guardrails/validator_service/__init__.py:85: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(


The result for 2+2 is 4. 2+2 is a simple mathematical equation that involves adding two numbers together. In this case, the two numbers are both 2. When you add 2 and 2 together, the result is 4. This is a basic arithmetic calculation that can be easily solved by most people. Is there anything else you would like to know about this equation?
